# Cleaning Data: Missing Data and Dealing with Outliers

![Data Science Workflow](img/ds-workflow.png)

## Cleaning Data
- Understand the **data quality**
- **Improve** the quality (if possible)
- Dealing with **missing data** (both rows in single entries)
    - Examples include 
        - **Replacing** missing values/entries with mean values
        - **Interpolation** of values (in time series)
- Dealing with **data outliers**
    - Examples include 
        - Default missing values in system: sometimes as 0-values
        - Wrong values
- Removing **duplicates**
    - Common problem to have duplicate entries
- Process requires **domain knowledge**

## Missing Data
- Missing data is sometimes refered to as **NA** values in pandas.
- A great source to learn about is [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html)


- Two types of missing data we consider
    1. NaN data
    2. Rows in time series data

Examples

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'a': [np.nan, 2, 3], 'b': [4, 5, np.nan]})
df

,a,b
0,NaN,4.0
1,2.0,5.0
2,3.0,NaN


In [4]:
df = pd.DataFrame([i for i in range(10)], columns=['Data'], index=pd.date_range("2021-01-01", periods=10))
df = df.drop(['2021-01-03', '2021-01-05', '2021-01-06'])
df

,Data
2021-01-01,0
2021-01-02,1
2021-01-04,3
2021-01-07,6
2021-01-08,7
2021-01-09,8
2021-01-10,9


## Outliers
- Requires domain knowledge
- But typical examples could include

In [5]:
df = pd.DataFrame({'Weight (kg)': [86, 83, 0, 76, 109, 95, 0]})
df

,Weight (kg)
0,86
1,83
2,0
3,76
4,109
5,95
6,0


## Demonstration how it affects results
- [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course/overview)
- The dataset contains a training and testing dataset.
    - The goal is to predict prices on the testing dataset.
- We will explore how dealing with missing values impacts the prediction of a linear regression model

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/LearnPythonWithRune/DataScienceWithPython/main/jupyter/final/files/home-data/train.csv', index_col=0)
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [8]:
data.shape


(1460, 80)

### Remove non-numeric for this demonstration
```Python
data = data.select_dtypes(include='number')
```

Then we check of missing values.
```Python
data.info()
```

In [9]:
data = data.select_dtypes(include='number')


In [10]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   OverallQual    1460 non-null   int64  
 4   OverallCond    1460 non-null   int64  
 5   YearBuilt      1460 non-null   int64  
 6   YearRemodAdd   1460 non-null   int64  
 7   MasVnrArea     1452 non-null   float64
 8   BsmtFinSF1     1460 non-null   int64  
 9   BsmtFinSF2     1460 non-null   int64  
 10  BsmtUnfSF      1460 non-null   int64  
 11  TotalBsmtSF    1460 non-null   int64  
 12  1stFlrSF       1460 non-null   int64  
 13  2ndFlrSF       1460 non-null   int64  
 14  LowQualFinSF   1460 non-null   int64  
 15  GrLivArea      1460 non-null   int64  
 16  BsmtFullBath   1460 non-null   int64  
 17  BsmtHalfBath   1460 non-null   int64  
 18  FullBath

In [11]:
data.corr()['SalePrice'].sort_values(ascending=False)


SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePrice, dtype: float64

### Helper function
- Implement a helper function to calculate the r-square score
- It should take independent features `X` and dependent feature `y`
- Then split that into training and testing sets.
- Fit the training set.
- Predict the test set.
- Return the r-square score

In [13]:
def regression_score(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    lin = LinearRegression()
    lin.fit(X_train, y_train)
    y_pred = lin.predict(X_test)
    return r2_score(y_pred, y_test)

### Calculations
- Try first to calcualte the r-square by using `data.dropna()`
    - This serves as the ussual way we have done it
- Then with `data.fillna(data.mean())`
    - [`fillna()`](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.fillna.html) Fill NA/NaN values using the specified method.
- Then with `data.fillna(data.mode().iloc[0])`

In [14]:
test_base = data.dropna()

regression_score(test_base.drop('SalePrice', axis=1), test_base[['SalePrice']])

0.6548289068325821

In [15]:
test_base = data.fillna(data.mean())

regression_score(test_base.drop('SalePrice', axis=1), test_base[['SalePrice']])

0.7439346373898352

In [16]:
data.mode()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,20,60.0,7200,5,5,2006,1950,0.0,0,0,...,0,0,0,0,0,0,0,6,2009,140000


In [17]:
test_base = data.fillna(data.mode().iloc[0])

regression_score(test_base.drop('SalePrice', axis=1), test_base[['SalePrice']])

0.7465420669734038

## Time series
- Now let's explore a time series: `files/weather.parquet`


In [19]:
weather = pd.read_parquet('files/weather.parquet')
weather.head()

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

### Missing time series rows
- One way to find missing rows of data in a time series is as follows
```Python
idx = pd.Series(data=pd.date_range(start=df.index.min(), end=df.index.max(), freq="H"))
mask = idx.isin(df.index)
idx[~mask]
```

In [ ]:
idx = pd.Series(data=pd.date_range(start=weather.index.min(), end=weather.index.max(), freq="H"))


In [ ]:
mask = idx.isin(weather.index)


In [ ]:
idx[~mask]


In [ ]:
weather.loc['2006-03-26 02:00:00']


### Insert missing datetimess and interpolate them
- To insert missing datetimes we can use [`reindex()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reindex.html)
- To interploate values that are missing [`interpolate`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html)

In [ ]:
w_idx = weather.reindex(idx)


In [ ]:
w_idx[w_idx['Summary'].isna()]


In [ ]:
w_idx.interpolate()[w_idx['Summary'].isna()]


## Outliers
- If we focus on `Pressure (millibars)` for `2006'
- One way to handle 0-values is with [`replace()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)
```Python
.replace(0, np.nan)
```
- Then we can apply `interploate()`

In [ ]:
(weather['Pressure (millibars)'].loc['2006']).isna().any()


In [ ]:
p_2016 = weather['Pressure (millibars)'].loc['2006']


In [ ]:
p_2016.plot()


In [ ]:
p_2016.isna().any()


In [ ]:
(p_2016 == 0).any()


In [ ]:
p_2016.replace(0, np.nan).interpolate().plot()


### Removing Duplicates
- [`drop_duplicates()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html) Return DataFrame with duplicate rows removed.

In [ ]:
df = pd.DataFrame({'a': [1, 2, 3, 2], 'b': [11, 2, 21, 2], 'c': [21, 2, 31, 2]})
df

In [ ]:
df.drop_duplicates()
